In [1]:
import torch
from datasets import load_dataset


#Get the data from hugginface
class Dataset(torch.utils.data.Dataset):
    def __init__(self, split):
        dataset = load_dataset(path='lansinuote/ChnSentiCorp', split=split)

        def f(data):
            return len(data['text']) > 30
        
        # only keep the senten length > 30
        self.dataset = dataset.filter(f)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['text']

        return text


dataset = Dataset('train')

# original dataset size 9600
len(dataset), dataset[0]

Found cached dataset parquet (C:/Users/35391/.cache/huggingface/datasets/lansinuote___parquet/lansinuote--ChnSentiCorp-4d058ef86e3db8d5/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
Loading cached processed dataset at C:\Users\35391\.cache\huggingface\datasets\lansinuote___parquet\lansinuote--ChnSentiCorp-4d058ef86e3db8d5\0.0.0\14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7\cache-1a622b13dfce1ac5.arrow


(9192,
 '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般')

In [2]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# use to(device) set the data and model to use cuda
device

device(type='cuda', index=0)

In [3]:
from transformers import BertTokenizer

# get the chinese version tokenizer
token = BertTokenizer.from_pretrained('bert-base-chinese')

token

BertTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [8]:
def collate_fn(data):
    # encode the data
    data = token.batch_encode_plus(batch_text_or_text_pairs=data,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=30, # set the sentence to 30 length
                                   return_tensors='pt',
                                   return_length=True)

    #input_ids:map the id to different words
    #attention_mask: words == 1, padding  == 0 [1,1,1,1,1,0,0]
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    
    # use for 1st sentence and 2nd sentence [0,0,0,0,0,1,1,1,1,1,1,1]
    token_type_ids = data['token_type_ids']

    # get the label ids which is the 15th word of the sentence
    labels = input_ids[:, 15].reshape(-1).clone()
#     print("labels")
#     print(labels)
    
    
    # token.mask_token return "[MASK]"
    # token.get_vocab() return the dict of all word:id in the tokenizer
    # token.get_vocab()[token.mask_token] return the "[MASK]" key value which is 103
    # {...'[SEP]': 102, '[MASK]': 103, '<S>': 104,....}
    # input_ids[:, 15] this will set all the 15th word of the sentence to be [MASK]/103
    input_ids[:, 15] = token.get_vocab()[token.mask_token]

    #print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels


# put the data into torch dataloader
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    break

print(len(loader))
print(token.decode(input_ids[0]))
print(token.decode(labels[0]))
input_ids.shape, attention_mask.shape, token_type_ids.shape, labels.shape

574
[CLS] 房 间 设 施 陈 旧 ， 气 味 太 重 ， 床 单 [MASK] 有 黄 色 斑 点 。 免 费 注 册 网 站 导 [SEP]
上


(torch.Size([16, 30]),
 torch.Size([16, 30]),
 torch.Size([16, 30]),
 torch.Size([16]))

In [11]:
from transformers import BertModel

# get the pretrained model of bert, need the same version of the tokenizer
pretrained = BertModel.from_pretrained('bert-base-chinese').to(device)

# don't need train the model and update the params
for param in pretrained.parameters():
    param.requires_grad_(False)

# try the first batch, last_hidden_state will give the model's output
out = pretrained(input_ids=input_ids.to(device),
           attention_mask=attention_mask.to(device),
           token_type_ids=token_type_ids.to(device))

out.last_hidden_state.shape

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([16, 30, 768])

In [16]:
# define forward model
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # set the Fully Connected Layer(fc layer)
        # output willbe the softmax(token.vocab_size) which size should be 21128 in this tokenizer
        self.decoder = torch.nn.Linear(768, token.vocab_size, bias=False)
        self.bias = torch.nn.Parameter(torch.zeros(token.vocab_size))
        self.decoder.bias = self.bias

    def forward(self, input_ids, attention_mask, token_type_ids):
        # do not need update the weight
        with torch.no_grad():
            out = pretrained(input_ids=input_ids.to(device),
                             attention_mask=attention_mask.to(device),
                             token_type_ids=token_type_ids.to(device))
        # get the softmax value of the 15th word
        out = self.decoder(out.last_hidden_state[:, 15])

        return out


model = Model().to(device)

model(input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids=token_type_ids).shape


torch.Size([16, 21128])

In [18]:
from transformers import AdamW

# train optimizer = AdamW
optimizer = AdamW(model.parameters(), lr=5e-4)
# define the loss
criterion = torch.nn.CrossEntropyLoss()

model.train()
for epoch in range(5):
    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader):
        out = model(input_ids=input_ids.to(device),
                    attention_mask=attention_mask.to(device),
                    token_type_ids=token_type_ids.to(device))

        loss = criterion(out, labels.to(device))
        # backforward
        loss.backward()
        # use the optimizer
        optimizer.step()
        # reset the optimizer step to zero, can be put in front of backward
        optimizer.zero_grad()

        if i % 50 == 0:
            # softmax => get the max value index
            out = out.argmax(dim=1)
            accuracy = (out == labels.to(device)).sum().item() / len(labels.to(device))

            print(epoch, i, loss.item(), accuracy)

0 0 10.03921127319336 0.0
0 50 8.445544242858887 0.125
0 100 5.90747594833374 0.125
0 150 5.278145790100098 0.1875
0 200 4.775643348693848 0.25
0 250 5.233343601226807 0.4375
0 300 3.9224624633789062 0.3125
0 350 3.90852427482605 0.5
0 400 2.863471746444702 0.5
0 450 4.3009843826293945 0.25
0 500 3.597964286804199 0.5
0 550 5.555667400360107 0.375
1 0 2.691502094268799 0.6875
1 50 2.3754396438598633 0.8125
1 100 2.2875373363494873 0.625
1 150 2.842221975326538 0.625
1 200 3.339266300201416 0.5
1 250 1.8524198532104492 0.6875
1 300 1.7356739044189453 0.5625
1 350 2.6079063415527344 0.5625
1 400 2.120748281478882 0.6875
1 450 2.364314079284668 0.625
1 500 2.830188512802124 0.5625
1 550 1.8289223909378052 0.5625
2 0 1.304487943649292 0.6875
2 50 0.4457993805408478 0.9375
2 100 0.901310920715332 0.875
2 150 1.272574782371521 0.875
2 200 1.1237680912017822 0.75
2 250 1.324442982673645 0.8125
2 300 0.8867257833480835 0.9375
2 350 0.7476117014884949 0.9375
2 400 0.7557695508003235 0.9375
2 45

In [7]:
# test the model
def test():
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=Dataset('test'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader_test):

        if i == 15:
            break

        print(i)

        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)

        out = out.argmax(dim=1)
        correct += (out == labels).sum().item()
        total += len(labels)

        print(token.decode(input_ids[0]))
        print(token.decode(labels[0]), token.decode(labels[0]))

    print(correct / total)


test()

Using custom data configuration lansinuote--ChnSentiCorp-4d058ef86e3db8d5
Reusing dataset parquet (/root/.cache/huggingface/datasets/lansinuote___parquet/lansinuote--ChnSentiCorp-4d058ef86e3db8d5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?ba/s]

0
[CLS] 床 发 出 吱 嘎 吱 嘎 的 声 音 ， 房 间 隔 [MASK] 太 差 ， 赠 送 的 早 餐 非 常 好 吃 。 [SEP]
音 音
1
[CLS] 非 常 不 好 ， 我 们 渡 过 了 一 个 让 人 [MASK] 以 忍 受 的 纪 念 日. com / thread - 136 [SEP]
难 难
2
[CLS] 定 的 商 务 大 床 房 ， 房 间 偏 小 了 ， [MASK] 过 经 济 性 酒 店 也 就 这 样 ； 环 境 [SEP]
不 不
3
[CLS] 确 实 是 山 上 最 好 的 酒 店 ， 环 境 和 [MASK] 施 都 很 不 错 。 我 们 这 次 住 的 是 [SEP]
设 设
4
[CLS] 这 本 书 紧 接 《 春 秋 大 义 》 ， 作 者 [MASK] 以 贯 之 地 以 浅 显 的 语 言 ， 告 诉 [SEP]
一 一
5
[CLS] 非 常 不 满 这 酒 店 ， 配 不 上 5 星 。 [MASK] 一, 客 房 服 务 员 没 有 水 平, 房 [SEP]
第 第
6
[CLS] 合 庆 的 商 务 单 间 可 以 堪 称 豪 华, [MASK] 施 特 别 先 进 ， 特 别 是 少 有 的 先 [SEP]
设 设
7
[CLS] 这 是 我 住 过 的 最 差 的 酒 店 ， 房 间 [MASK] 味 难 闻 ， 刚 打 了 灭 蚊 药 水 ， 换 [SEP]
气 气
8
[CLS] 总 体 很 满 意 ， 但 有 一 点 需 改 进 ， [MASK] 在 9 楼 入 住 ， 走 时 到 1 楼 前 台 [SEP]
我 我
9
[CLS] 这 本 书 有 别 于 以 往 看 过 的 早 教 书 [MASK] ， 结 合 了 说 明 文 的 写 实 ， 散 文 [SEP]
籍 籍
10
[CLS] [UNK] 用 起 来 不 习 惯 ， 速 度 慢 ， 分 区 [MASK] 烦 ， 带 了 很 多 垃 圾 软 件 ， 卸 载 [SEP]
麻 麻
11
[CLS] 这 一 套 书 我 基 本 买 齐 了 ， 也 看 了 [MASK] 多 本 了 。 是 利 用 闲 暇 时 间 巩 固 [SEP]
好 好
12
[CLS] 渡 假 村 周 围 景 色 不 错, 但 较 落 乡 [MASK